In [ ]:
import os
import logging
import osmnx as ox
import numpy as np
import pandas as pd
import geopandas as gpd
import h3
import shapely
import requests
import plotly.express as px
from tqdm.notebook import tqdm
from pyproj import Transformer
from shapely.ops import linemerge
from shapely import Polygon, Point
from pyogrio.errors import DataSourceError

ox.settings.cache_folder = './data'

## functions

In [ ]:
def fetch_google_places(categories:list[str], lon:float, lat:float, radius:float, apikey:str):
    
    max_results = 20

    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": apikey,
        "X-Goog-FieldMask": "places.id,places.displayName,places.location,places.businessStatus,places.formattedAddress,places.primaryTypeDisplayName,places.primaryType",
    }
    url = "https://places.googleapis.com/v1/places:searchNearby"

    data = {
            "includedTypes": categories,
            "maxResultCount": max_results,
            "locationRestriction": {
                "circle": {
                    "center": {"latitude": lat, "longitude": lon},
                    "radius": radius,
                }
            },
        }
    
    r = requests.post(url, headers=headers, json=data)
    if r.status_code != 200:
        logging.warning(f"error at request: {r.request}")
        return

    data = r.json().get('places', [])
    if not data:
        return data

    if len(data) >= max_results:
        logging.warning(f'query at lon:{lon}, lat:{lat} returned 20+ results')
    
    for r in data:
        r['name'] = r['displayName']['text']
        r['geometry'] = Point(r['location']['longitude'], r['location']['latitude'])

    return data



In [ ]:
def h3cell_to_shapely_polygon(cell:str):
    coords = h3.cell_to_boundary(cell)
    flipped = tuple(coord[::-1] for coord in coords)
    return Polygon(flipped)

## inputs

In [ ]:
apikey = os.getenv("GOOGLE_API_KEY")
radius = 200
step = 60
buffer = 30
categories = ['cafe', 'bakery']
map_style = "open-street-map"

# Geospatial Analysis

## download network (roads)

In [ ]:
bbox = [23.7576, 37.8524, 23.7729, 37.899]
g = ox.graph_from_bbox(bbox) # we can modify filters
nodes, links = ox.graph_to_gdfs(g)

## create the linestring geometry

In [ ]:
road = links[links['name'] == 'Δημητρίου Γούναρη']
crs = road.estimate_utm_crs()
geom_proj = road.to_crs(crs).union_all()
geoms_proj = gpd.GeoDataFrame(geometry=[linemerge(geom_proj)]).explode()
geoms_proj['length'] = geoms_proj.length
geom_proj = geoms_proj.sort_values(['length'], ascending=False).iloc[0].geometry
road_buffered = road.to_crs(crs).buffer(buffer).to_crs(epsg=4326).union_all()

In [ ]:
fp = './data/cafes.gpkg'
length = geom_proj.length

try:
    gdf = gpd.read_file(fp)
except DataSourceError:
    places = []
    transformer = Transformer.from_crs(crs, "EPSG:4326", always_xy=True)
    for distance in tqdm(np.arange(0, length, step)):
        p = shapely.line_interpolate_point(geom_proj, distance) # type:ignore
        p = transformer.transform(p.x, p.y)
        p = shapely.Point(p[0], p[1])
        r = fetch_google_places(categories, p.x, p.y, radius, apikey)
        if r is not None:
            places.extend(r)

    gdf = gpd.GeoDataFrame(places, crs='epsg:4326').drop_duplicates(subset=['id'])
    gdf['inbuffer'] = gdf.intersects(road_buffered) # keep cafes up to 30 meters from the road (inside the buffer)
    gdf.to_file(fp)    

In [ ]:
road

In [ ]:
gdf['inbuffer'] = gdf.intersects(road_buffered) # keep cafes up to 30 meters from the road (inside the buffer)
cafes = gdf[gdf['inbuffer']]
cafes.head(2)

## stats / visuals

In [ ]:
fig = px.scatter_map(gdf, lat=gdf.geometry.y, lon=gdf.geometry.x, color='inbuffer', text='name', hover_name="name", zoom=12, map_style=map_style)
fig.show()

In [ ]:
fig = px.density_map(cafes, lat=cafes.geometry.y, lon=cafes.geometry.x, radius=12, zoom=12, map_style=map_style, width=600)
fig.show()

In [ ]:
f'One cafe every: {length / cafes.shape[0]:.0f}m'

### Distance between cafes (distance matrix)

In [ ]:
gdf = cafes.to_crs(crs) # project the coordinates

dists = []
for i, g in gdf.iterrows():
    geom = g.geometry
    d = gdf.distance(geom).tolist()
    dists.append(d)

dists = pd.DataFrame(dists, index=gdf.index, columns=gdf.index).round(2)
dists

In [ ]:
nearests = gdf.sjoin_nearest(gdf, distance_col='sdist', exclusive=True)[['id_left', 'id_right', 'sdist']]
nearests['sdist'].mean()

In [ ]:
cafes[cafes.id.isin(['ChIJgcEWYU6_oRQR-cd2bf6di68', 'ChIJ78DvEB2-oRQRJRLxgttGeas'])]

### spatial grouping (h3)

In [ ]:
box = shapely.box(*bbox)
coords = [list(box.exterior.coords)]
cells = h3.geo_to_cells(box, res=10)
cells = [{'h3id': cell, 'geometry': h3cell_to_shapely_polygon(cell)} for cell in cells]
cells = gpd.GeoDataFrame(cells, crs='epsg:4326')
cells.head(2)
px.choropleth_map(cells, geojson=cells.geometry, locations=cells.index,
                  center={'lat': box.centroid.y, 'lon': box.centroid.x}, zoom=11.5,
                  map_style=map_style)

In [ ]:
counts = gpd.sjoin(cells, cafes, predicate='contains').groupby(['h3id', 'geometry'], as_index=False)['index_right'].count()
counts = counts.rename(columns={'index_right':'count'})
counts = gpd.GeoDataFrame(counts) # groupby casts to pd.DataFrame
px.choropleth_map(counts, geojson=counts.geometry, locations=counts.index,
                  color='count', color_continuous_scale=px.colors.sequential.Reds,
                  center={'lat': box.centroid.y, 'lon': box.centroid.x}, zoom=11.5,
                  title='Concentration of cafes (H3 cells)', map_style=map_style)